In [ ]:
modules = ["neo4j", "pprintpp", "rich","tqdm"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")  
          try:        
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")
              
  print("¡Terminado!")

instala(modules) 

In [ ]:
from neo4j import GraphDatabase
import csv

uri = "bolt://localhost:example"
user = "neo4j"
password = "password"

driver = GraphDatabase.driver(uri, auth=(user, password))

query = """
MATCH (n)
WHERE 'DEM' IN labels(n) OR 'REP' IN labels(n)
RETURN
  n.user_id AS userid,
  n.screen AS screen,
  n.created_at AS created_at,
  n.common_degree AS common_degree,
  n.verified AS verified,
  n.followers AS followers,
  n.following AS following,
  n.nTweets AS nTweets,
  CASE 
    WHEN n.contagio is not null THEN 'contagio'
    WHEN n.manual is not null THEN 'manual'
    ELSE null
  END AS sent,
  CASE 
    WHEN 'DEM' IN labels(n) THEN 'DEM'
    WHEN 'REP' IN labels(n) THEN 'REP'
  END AS tipo
"""

def name(path,year):
    return path+"neo4j_nodos_"+str(year)+".csv"
path = "../data/"
year =  24
with driver.session() as session:
    result = session.run(query)
    file = name(path,year)
    with open(file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(result.keys())  # headers
        for row in result:
            writer.writerow(row.values())

driver.close()

In [ ]:
import pandas as pd
df = pd.read_csv(file)
df

Load and process files

In [ ]:
import pandas as pd
def name(path,year):
    return path+"neo4j_nodos_"+str(year)+".csv"
path = "../data/"
years = [16,20,24]
for i,year in enumerate(years):
    file = name(path,year)
    df = pd.read_csv(file)
    if i==0:
        union = df
    else:
        union = union.merge(df,left_on="userid",right_on="userid",suffixes=("","_"+str(year)))
    print(year,len(union))
    

In [ ]:
union

In [ ]:
cambio = (union.tipo != union.tipo_24) | (union.tipo_20 != union.tipo_24) | (union.tipo != union.tipo_20)
union[cambio][["screen","tipo","tipo_20","tipo_24"]]

In [ ]:
union[cambio][["screen","tipo","tipo_20","tipo_24"]].screen.values